stdin, stdout, stderr = client.exec_command()

print_std(stdout, stderr)

In [ ]:
import paramiko, json

with open("../01-deploy-aws-infra/inventory.json", 'r') as file:
    _DATA = json.load(file)
_SSH_KEY = _DATA["KeyPairPath"]
_HOST = _DATA["Instances"][0]["InstanceIp"]
_PORT = 22
_USER = "ubuntu"

In [ ]:
def print_std(stdout, stderr):
    stdout = stdout.readlines()
    stderr = stderr.readlines()
    print("\t*** STDOUT ***")
    [print(i) for i in stdout]
    print("\n\t*** STDERR ***")
    [print(i) for i in stderr]
    
def get_stdout(stdout):
    stdout = stdout.readline()
    return stdout

# Config Master + Slave

In [ ]:
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname=_HOST, username=_USER, key_filename=_SSH_KEY)





In [ ]:
stdin, stdout, stderr = client.exec_command("swapoff -a")
print_std(stdout, stderr)

In [ ]:
stdin, stdout, stderr = client.exec_command("hostname -i")
host_ip = get_stdout(stdout)
print(host_ip)


## Installation Docker

In [ ]:
stdin, stdout, stderr = client.exec_command("wget -qO- https://get.docker.com/ | sh")
print_std(stdout, stderr)

## Installation Kubernetes

In [ ]:
stdin, stdout, stderr = client.exec_command("modprobe br_netfilter")
print_std(stdout, stderr)

stdin, stdout, stderr = client.exec_command("""
cat <<EOF | sudo tee /etc/sysctl.d/k8s.conf
net.bridge.bridge-nf-call-ip6tables = 1
net.bridge.bridge-nf-call-iptables = 1 
EOF""")
print_std(stdout, stderr)


In [ ]:
stdin, stdout, stderr = client.exec_command("sudo sysctl --system")
print_std(stdout, stderr)

In [ ]:
stdout, stdin, stderr = client.exec_command("curl -s https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -")
print_std(stdin, stderr)

stdout, stdin, stderr = client.exec_command("""
cat <<EOF | sudo tee /etc/apt/sources.list.d/kubernetes.list
deb https://apt.kubernetes.io/ kubernetes-xenial main
EOF""")
print_std(stdin, stderr)

In [ ]:
stdout, stdin, stderr = client.exec_command("sudo apt-get update")
print_std(stdin, stderr)

stdout, stdin, stderr = client.exec_command("sudo apt-get install -y kubelet kubeadm kubectl")
print_std(stdin, stderr)

In [ ]:
stdin, stdout, stderr = client.exec_command("sudo sed -i 's|Environment=\"KUBELET_CONFIG_ARGS=--config=/var/lib/kubelet/config.yaml\"|Environment=\"KUBELET_CONFIG_ARGS=--config=/var/lib/kubelet/config.yaml --cgroup-driver=cgroupfs\"|' /etc/systemd/system/kubelet.service.d/10-kubeadm.conf")
print_std(stdout, stderr)

In [ ]:
stdin, stdout, stderr = client.exec_command("sudo systemctl daemon-reload")
print_std(stdout, stderr)

In [ ]:
client.close()

# Configuration Master

In [ ]:
client.connect(hostname=_HOST, username=_USER, key_filename=_SSH_KEY)


In [ ]:
stdin, stdout, stderr = client.exec_command("sudo rm /etc/containerd/config.toml && sudo systemctl restart containerd")
print_std(stdout, stderr)

In [ ]:
stdin, stdout, stderr = client.exec_command("sudo kubeadm init --pod-network-cidr=10.244.0.0/16 --ignore-preflight-errors=all --v=5")
print_std(stdout, stderr)

In [ ]:
stdin, stdout, stderr = client.exec_command("sudo kubeadm token create --print-join-command")
master_join_command = get_stdout(stdout)
print(master_join_command)

In [ ]:
stdin, stdout, stderr = client.exec_command("sudo mkdir -p $HOME/.kube && sudo cp -i /etc/kubernetes/admin.conf $HOME/.kube/config && sudo chown $(id -u):$(id -g) $HOME/.kube/config")
print_std(stdout, stderr)

In [ ]:
stdin, stdout, stderr = client.exec_command("kubectl get nodes && kubectl get pods --all-namespaces")
print_std(stdout, stderr)

In [ ]:
stdin, stdout, stderr = client.exec_command("kubectl apply -f https://raw.githubusercontent.com/coreos/flannel/master/Documentation/kube-flannel.yml")
print_std(stdout, stderr)

In [ ]:
stdin, stdout, stderr = client.exec_command("kubectl get nodes && kubectl get pods --all-namespaces")
print_std(stdout, stderr)